# Capstone Project
## Neural translation model
### Instructions

In this notebook, you will create a neural network that translates from English to German. You will use concepts from throughout this course, including building more flexible model architectures, freezing layers, data processing pipeline and sequence modelling.

This project is peer-assessed. Within this notebook you will find instructions in each section for how to complete the project. Pay close attention to the instructions as the peer review will be carried out according to a grading rubric that checks key parts of the project instructions. Feel free to add extra cells into the notebook as required.

### How to submit

When you have completed the Capstone project notebook, you will submit a pdf of the notebook for peer review. First ensure that the notebook has been fully executed from beginning to end, and all of the cell outputs are visible. This is important, as the grading rubric depends on the reviewer being able to view the outputs of your notebook. Save the notebook as a pdf (File -> Download as -> PDF via LaTeX). You should then submit this pdf for review.

### Let's get started!

We'll start by running some imports, and loading the dataset. For this project you are free to make further imports throughout the notebook as you wish. 

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import unicodedata
import re
import random
import numpy as np
import json

from sklearn.model_selection import train_test_split

![Flags overview image](data/germany_uk_flags.png)

For the capstone project, you will use a language dataset from http://www.manythings.org/anki/ to build a neural translation model. This dataset consists of over 200,000 pairs of sentences in English and German. In order to make the training quicker, we will restrict to our dataset to 20,000 pairs. Feel free to change this if you wish - the size of the dataset used is not part of the grading rubric.

Your goal is to develop a neural translation model from English to German, making use of a pre-trained English word embedding module.

In [3]:
# Run this cell to load the dataset

NUM_EXAMPLES = 20000
data_examples = []
with open('../../data/deu.txt', 'r', encoding='utf8') as f:
    for line in f.readlines():
        if len(data_examples) < NUM_EXAMPLES:
            data_examples.append(line)
        else:
            break

In [4]:
data_examples[:3]

['Hi.\tHallo!\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #380701 (cburgmer)\n',
 'Hi.\tGrüß Gott!\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #659813 (Esperantostern)\n',
 'Run!\tLauf!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #941078 (Fingerhut)\n']

In [5]:
# These functions preprocess English and German sentences

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"ü", 'ue', sentence)
    sentence = re.sub(r"ä", 'ae', sentence)
    sentence = re.sub(r"ö", 'oe', sentence)
    sentence = re.sub(r'ß', 'ss', sentence)
    
    sentence = unicode_to_ascii(sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
#     sentence = re.sub(r"([?.!,])", " ", sentence)
    sentence = re.sub(r"[^a-z?.!,']+", " ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    
    return sentence.strip()

#### The custom translation model
The following is a schematic of the custom translation model architecture you will develop in this project.

![Model Schematic](data/neural_translation_model.png)

Key:
![Model key](data/neural_translation_model_key.png)

The custom model consists of an encoder RNN and a decoder RNN. The encoder takes words of an English sentence as input, and uses a pre-trained word embedding to embed the words into a 128-dimensional space. To indicate the end of the input sentence, a special end token (in the same 128-dimensional space) is passed in as an input. This token is a TensorFlow Variable that is learned in the training phase (unlike the pre-trained word embedding, which is frozen).

The decoder RNN takes the internal state of the encoder network as its initial state. A start token is passed in as the first input, which is embedded using a learned German word embedding. The decoder RNN then makes a prediction for the next German word, which during inference is then passed in as the following input, and this process is repeated until the special `<end>` token is emitted from the decoder.

## 1. Text preprocessing
* Create separate lists of English and German sentences, and preprocess them using the `preprocess_sentence` function provided for you above.
* Add a special `"<start>"` and `"<end>"` token to the beginning and end of every German sentence.
* Use the Tokenizer class from the `tf.keras.preprocessing.text` module to tokenize the German sentences, ensuring that no character filters are applied. _Hint: use the Tokenizer's "filter" keyword argument._
* Print out at least 5 randomly chosen examples of (preprocessed) English and German sentence pairs. For the German sentence, print out the text (with start and end tokens) as well as the tokenized sequence.
* Pad the end of the tokenized German sequences with zeros, and batch the complete set of sequences into one numpy array.

In [6]:
data_examples[1]

'Hi.\tGrüß Gott!\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #659813 (Esperantostern)\n'

In [7]:
re.split('\t',data_examples[1])[0]

'Hi.'

In [8]:
re.split('\t',data_examples[1])[1]

'Grüß Gott!'

In [9]:
eng_sentences = []
ger_sentences = []

for sentence in data_examples:
    sentence = sentence.split('\t')
    
    eng_sentence = sentence[0]
    ger_sentence = sentence[1]
    
    eng_sentence = unicode_to_ascii(preprocess_sentence(eng_sentence))
    ger_sentence = unicode_to_ascii(preprocess_sentence(ger_sentence))
    
    ger_sentence = '<start>' + ger_sentence + '<end>'
    
    eng_sentences.append(eng_sentence)
    ger_sentences.append(ger_sentence)

In [10]:
len(ger_sentences)

20000

In [11]:
ger_sentences[:5]

['<start>hallo !<end>',
 '<start>gruess gott !<end>',
 '<start>lauf !<end>',
 '<start>potzdonner !<end>',
 '<start>donnerwetter !<end>']

In [12]:
eng_sentences[:5]

['hi .', 'hi .', 'run !', 'wow !', 'wow !']

In [13]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=[], char_level=False)

tokenizer.fit_on_texts(ger_sentences)

tokenizer_config = tokenizer.get_config()

word_indexs = json.loads(tokenizer_config['word_index'])
index_words = json.loads(tokenizer_config['index_word'])

In [14]:
print(len(tokenizer.word_index) == len(word_indexs))
print(len(word_indexs))

True
6459


In [15]:
tokenizer_config.keys()

dict_keys(['num_words', 'filters', 'lower', 'split', 'char_level', 'oov_token', 'document_count', 'word_counts', 'word_docs', 'index_docs', 'index_word', 'word_index'])

In [16]:
ger_sequences = tokenizer.texts_to_sequences(ger_sentences)

In [17]:
num_samples = 5
idxs = np.random.choice(len(ger_sentences), num_samples, replace=False)

# print(idxs)

for i, (e_sentence, g_sentence, g_seq) in enumerate(zip(np.array(eng_sentences)[idxs], \
                                                        np.array(ger_sentences)[idxs], \
                                                        np.array(ger_sequences)[idxs])):
    print(i, e_sentence, g_sentence)
    print(i, g_sentence, g_seq)

0 go away . <start>verzieh dich !<end>
0 <start>verzieh dich !<end> [1005, 30, 6]
1 can i sing ? <start>kann ich singen ?<end>
1 <start>kann ich singen ?<end> [141, 29, 340, 5]
2 you know him . <start>du kennst ihn .<end>
2 <start>du kennst ihn .<end> [28, 1053, 44, 1]
3 may i go home ? <start>darf ich nach hause gehen ?<end>
3 <start>darf ich nach hause gehen ?<end> [161, 29, 67, 72, 52, 5]
4 i'm color blind . <start>ich bin farbenblind .<end>
4 <start>ich bin farbenblind .<end> [2, 11, 4853, 1]


C:\Users\hewar\AppData\Local\Temp/ipykernel_8352/1180864545.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(ger_sequences)[idxs])):


In [18]:
len_sequences = []
for i in range(len(ger_sequences)):
    len_sequences += [len(ger_sequences[i])]
print('max length: ', max(len_sequences))

max length:  12


In [19]:
ger_sequences_pad = tf.keras.preprocessing.sequence.pad_sequences(ger_sequences,
                                                                  padding='post')
ger_sequences_pad = np.asarray(ger_sequences_pad)

In [20]:
print(ger_sequences_pad.shape)

(20000, 12)


## 2. Prepare the data with tf.data.Dataset objects

#### Load the embedding layer
As part of the dataset preproceessing for this project, you will use a pre-trained English word embedding module from TensorFlow Hub. The URL for the module is https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1. This module has also been made available as a complete saved model in the folder `'./models/tf2-preview_nnlm-en-dim128_1'`. 

This embedding takes a batch of text tokens in a 1-D tensor of strings as input. It then embeds the separate tokens into a 128-dimensional space. 

The code to load and test the embedding layer is provided for you below.

**NB:** this model can also be used as a sentence embedding module. The module will process each token by removing punctuation and splitting on spaces. It then averages the word embeddings over a sentence to give a single embedding vector. However, we will use it only as a word embedding module, and will pass each word in the input sentence as a separate token.

In [21]:
# Load embedding module from Tensorflow Hub

embedding_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1",
                                 output_shape=[128], input_shape=[], dtype=tf.string)

# embedding_layer = tf.keras.models.load_model('./models/tf2-preview_nnlm-en-dim128_1')

ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine

In [ ]:
# Test the layer

embedding_layer(tf.constant(["these", "aren't", "the", "droids", "you're", "looking", "for"])).shape

You should now prepare the training and validation Datasets.

* Create a random training and validation set split of the data, reserving e.g. 20% of the data for validation (NB: each English dataset example is a single sentence string, and each German dataset example is a sequence of padded integer tokens).
* Load the training and validation sets into a tf.data.Dataset object, passing in a tuple of English and German data for both training and validation sets.
* Create a function to map over the datasets that splits each English sentence at spaces. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.strings.split function._
* Create a function to map over the datasets that embeds each sequence of English words using the loaded embedding layer/model. Apply this function to both Dataset objects using the map method.
* Create a function to filter out dataset examples where the English sentence is more than 13 (embedded) tokens in length. Apply this function to both Dataset objects using the filter method.
* Create a function to map over the datasets that pads each English sequence of embeddings with some distinct padding value before the sequence, so that each sequence is length 13. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.pad function. You can extract a Tensor shape using tf.shape; you might also find the tf.math.maximum function useful._
* Batch both training and validation Datasets with a batch size of 16.
* Print the `element_spec` property for the training and validation Datasets. 
* Using the Dataset `.take(1)` method, print the shape of the English data example from the training Dataset.
* Using the Dataset `.take(1)` method, print the German data example Tensor from the validation Dataset.

In [23]:
eng_train, eng_val, ger_train, ger_val = train_test_split(eng_sentences, ger_sequences_pad, test_size=0.2)

In [24]:
eng_train[0]

'do you deny it ?'

In [25]:
train_dataset = tf.data.Dataset.from_tensor_slices((eng_train, ger_train))
val_dataset = tf.data.Dataset.from_tensor_slices((eng_val, ger_val))

In [26]:
print(train_dataset.element_spec)
print(val_dataset.element_spec)

(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(12,), dtype=tf.int32, name=None))
(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(12,), dtype=tf.int32, name=None))


In [28]:
tf.strings.split(eng_train[0], ' ')

<tf.Tensor: id=246, shape=(5,), dtype=string, numpy=array([b'do', b'you', b'deny', b'it', b'?'], dtype=object)>

In [29]:
def map_split(eng,ger):
    inp = []
    inp.append(tf.strings.split(eng,' '))
    inp.append(ger)
    return inp[0],inp[1]

In [30]:
train_dataset = train_dataset.map(map_split)
val_dataset = val_dataset.map(map_split)

In [31]:
print(train_dataset.element_spec)
print(val_dataset.element_spec)

(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(12,), dtype=tf.int32, name=None))
(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(12,), dtype=tf.int32, name=None))


In [34]:
def map_embeddings(eng, ger):
    inp = []
    inp.append(embedding_layer(eng))
    inp.append(ger)
    return inp[0],inp[1]

In [35]:
train_dataset = train_dataset.map(map_embeddings)
val_dataset = val_dataset.map(map_embeddings)

In [36]:
print(train_dataset.element_spec)
print(val_dataset.element_spec)

(TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(12,), dtype=tf.int32, name=None))
(TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(12,), dtype=tf.int32, name=None))


In [40]:
def map_filter(dataset):
    return dataset.filter(lambda eng, ger: len(eng) < 14)

In [41]:
train_dataset = train_dataset.apply(map_filter)
val_dataset = val_dataset.apply(map_filter)
# train_dataset = train_dataset.filter(lambda x,y: False if tf.shape(x)[0] > 13 else True)

In [42]:
print(train_dataset.element_spec)
print(val_dataset.element_spec)

(TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(12,), dtype=tf.int32, name=None))
(TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(12,), dtype=tf.int32, name=None))


In [43]:
def map_pad(eng, ger):
    paddings = [[13,0], [0,0]]
    eng_ = tf.pad(eng, paddings)
    return eng_[-13:,:], ger

In [46]:
train_dataset = train_dataset.map(map_pad)
val_dataset = val_dataset.map(map_pad)

In [47]:
print(train_dataset.element_spec[0].shape)
print(val_dataset.element_spec[0].shape)

(None, 128)
(None, 128)


In [48]:
train_dataset = train_dataset.batch(16, drop_remainder=True)
val_dataset = val_dataset.batch(16, drop_remainder=True)

In [51]:
print(train_dataset.element_spec)
print(val_dataset.element_spec)

(TensorSpec(shape=(16, None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(16, 12), dtype=tf.int32, name=None))
(TensorSpec(shape=(16, None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(16, 12), dtype=tf.int32, name=None))


In [49]:
print(train_dataset.element_spec[0].shape)
print(val_dataset.element_spec[0].shape)

(16, None, 128)
(16, None, 128)


## 3. Create the custom layer
You will now create a custom layer to add the learned end token embedding to the encoder model:

![Encoder schematic](data/neural_translation_model_encoder.png)

You should now build the custom layer.
* Using layer subclassing, create a custom layer that takes a batch of English data examples from one of the Datasets, and adds a learned embedded ‘end’ token to the end of each sequence. 
* This layer should create a TensorFlow Variable (that will be learned during training) that is 128-dimensional (the size of the embedding space). _Hint: you may find it helpful in the call method to use the tf.tile function to replicate the end token embedding across every element in the batch._
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and print the shape. Test the custom layer by calling the layer on the English data batch Tensor and print the resulting Tensor shape (the layer should increase the sequence length by one).

In [52]:
class MyLayer(tf.keras.layers.Layer):
    def __init__(self, input_dim=[1,1,128], **kwargs):
        super(MyLayer, self).__init__(**kwargs)
        self.end_token = tf.Variable(initial_value=tf.random.uniform(shape=input_dim, dtype='float32'),
                                     trainable=True)
                                                             
    def call(self, inputs):
        end_token = self.end_token
        #tile_spec = tf.constant([tf.shape(inputs)[0],1,1])
        tiling = tf.tile(end_token, [tf.shape(inputs)[0],1,1])
        return tf.keras.layers.concatenate([inputs,tiling],axis=1)

In [53]:
end_token_layer = MyLayer()

In [54]:
train_dataset.take(1)

<TakeDataset shapes: ((16, None, 128), (16, 12)), types: (tf.float32, tf.int32)>

In [55]:
for eng, ger in train_dataset.take(1):
    print(tf.shape(eng))
    print(eng.shape)
    layer_test = end_token_layer(eng)
    print(layer_test.shape)

tf.Tensor([ 16  13 128], shape=(3,), dtype=int32)
(16, 13, 128)
(16, 14, 128)


## 4. Build the encoder network
The encoder network follows the schematic diagram above. You should now build the RNN encoder model.
* Using the functional API, build the encoder network according to the following spec:
    * The model will take a batch of sequences of embedded English words as input, as given by the Dataset objects.
    * The next layer in the encoder will be the custom layer you created previously, to add a learned end token embedding to the end of the English sequence.
    * This is followed by a Masking layer, with the `mask_value` set to the distinct padding value you used when you padded the English sequences with the Dataset preprocessing above.
    * The final layer is an LSTM layer with 512 units, which also returns the hidden and cell states.
    * The encoder is a multi-output model. There should be two output Tensors of this model: the hidden state and cell states of the LSTM layer. The output of the LSTM layer is unused.
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and test the encoder model by calling it on the English data Tensor, and print the shape of the resulting Tensor outputs.
* Print the model summary for the encoder network.

In [57]:
inputs = tf.keras.Input(batch_shape=(None, 13, 128))

h = MyLayer()(inputs)
h = tf.keras.layers.Masking(mask_value=0.0)(h)

LSTM_out, hidden_state, cell_state = tf.keras.layers.LSTM(units=512,
                                                          return_state = True,
                                                          return_sequences=True)(h)

encoder_model = tf.keras.Model(inputs = inputs, outputs= [hidden_state, cell_state])

In [58]:
for eng, ger in train_dataset.take(1):
    h_state, c_state = encoder_model(eng)
    print(h_state.shape)
    print(tf.shape(c_state))

(16, 512)
tf.Tensor([ 16 512], shape=(2,), dtype=int32)


In [59]:
encoder_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 13, 128)]         0         
_________________________________________________________________
my_layer_2 (MyLayer)         (None, 14, 128)           128       
_________________________________________________________________
masking (Masking)            (None, 14, 128)           0         
_________________________________________________________________
lstm (LSTM)                  [(None, 14, 512), (None,  1312768   
Total params: 1,312,896
Trainable params: 1,312,896
Non-trainable params: 0
_________________________________________________________________


## 5. Build the decoder network
The decoder network follows the schematic diagram below. 

![Decoder schematic](data/neural_translation_model_decoder.png)

You should now build the RNN decoder model.
* Using Model subclassing, build the decoder network according to the following spec:
    * The initializer should create the following layers:
        * An Embedding layer with vocabulary size set to the number of unique German tokens, embedding dimension 128, and set to mask zero values in the input.
        * An LSTM layer with 512 units, that returns its hidden and cell states, and also returns sequences.
        * A Dense layer with number of units equal to the number of unique German tokens, and no activation function.
    * The call method should include the usual `inputs` argument, as well as the additional keyword arguments `hidden_state` and `cell_state`. The default value for these keyword arguments should be `None`.
    * The call method should pass the inputs through the Embedding layer, and then through the LSTM layer. If the `hidden_state` and `cell_state` arguments are provided, these should be used for the initial state of the LSTM layer. _Hint: use the_ `initial_state` _keyword argument when calling the LSTM layer on its input._
    * The call method should pass the LSTM output sequence through the Dense layer, and return the resulting Tensor, along with the hidden and cell states of the LSTM layer.
* Using the Dataset `.take(1)` method, extract a batch of English and German data examples from the training Dataset. Test the decoder model by first calling the encoder model on the English data Tensor to get the hidden and cell states, and then call the decoder model on the German data Tensor and hidden and cell states, and print the shape of the resulting decoder Tensor outputs.
* Print the model summary for the decoder network.

In [62]:
# Build the model using custom layers with the model subclassing API
class MyDecoder(tf.keras.Model):
    def __init__(self):
        super(MyDecoder, self).__init__()
    
        # Define layers
        self.layer_1 = tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1,
                                                 output_dim=128,
                                                mask_zero=True,
                                                name='Embedding')
        
        self.layer_2= tf.keras.layers.LSTM(units=512,
                                           return_sequences=True,
                                           return_state=True,
                                          name='LSTM')
        
        self.layer_3 = tf.keras.layers.Dense(units=len(tokenizer.word_index) + 1,
                                            name='Dense')
                                       
    def call(self, inputs, hidden_state=None, cell_state=None):
        # Define forward pass
        h = self.layer_1(inputs)
        
        if (hidden_state is None) or (cell_state is None):
            h, hidden_s,cell_s = self.layer_2(h)
        else:
            h, hidden_s, cell_s = self.layer_2(h, initial_state = [hidden_state, cell_state])

        h = self.layer_3(h)
                                                                    
        return h, hidden_s, cell_s

In [63]:
decoder_model = MyDecoder()

for eng, ger in train_dataset.take(1):
    h_state, c_state = encoder_model(eng)
    print(tf.shape(h_state))
    print(tf.shape(c_state))

for eng, ger in train_dataset.take(1):
    h_g, h_s_g, c_s_g = decoder_model(ger, h_state, c_state)
    print(h_g.shape)
    print(h_s_g.shape)
    print(c_s_g.shape)

tf.Tensor([ 16 512], shape=(2,), dtype=int32)
tf.Tensor([ 16 512], shape=(2,), dtype=int32)
(16, 12, 6460)
(16, 512)
(16, 512)


In [64]:
decoder_model.summary()

Model: "my_decoder_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding (Embedding)        multiple                  826880    
_________________________________________________________________
LSTM (LSTM)                  multiple                  1312768   
_________________________________________________________________
Dense (Dense)                multiple                  3313980   
Total params: 5,453,628
Trainable params: 5,453,628
Non-trainable params: 0
_________________________________________________________________


## 6. Make a custom training loop
You should now write a custom training loop to train your custom neural translation model.
* Define a function that takes a Tensor batch of German data (as extracted from the training Dataset), and returns a tuple containing German inputs and outputs for the decoder model (refer to schematic diagram above).
* Define a function that computes the forward and backward pass for your translation model. This function should take an English input, German input and German output as arguments, and should do the following:
    * Pass the English input into the encoder, to get the hidden and cell states of the encoder LSTM.
    * These hidden and cell states are then passed into the decoder, along with the German inputs, which returns a sequence of outputs (the hidden and cell state outputs of the decoder LSTM are unused in this function).
    * The loss should then be computed between the decoder outputs and the German output function argument.
    * The function returns the loss and gradients with respect to the encoder and decoder’s trainable variables.
    * Decorate the function with @tf.function
* Define and run a custom training loop for a number of epochs (for you to choose) that does the following:
    * Iterates through the training dataset, and creates decoder inputs and outputs from the German sequences.
    * Updates the parameters of the translation model using the gradients of the function above and an optimizer object.
    * Every epoch, compute the validation loss on a number of batches from the validation and save the epoch training and validation losses.
* Plot the learning curves for loss vs epoch for both training and validation sets.

_Hint: This model is computationally demanding to train. The quality of the model or length of training is not a factor in the grading rubric. However, to obtain a better model we recommend using the GPU accelerator hardware on Colab._

In [66]:
def german_in_out(input_):
    return (input_[:,:-1], input_[:,1:])

In [67]:
# Create the optimizer and loss
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

trainable_variables = encoder_model.trainable_variables + decoder_model.trainable_variables

In [68]:
@tf.function

def grad_fn(eng, ger_in, ger_out):
    # h_state, c_state = encoder_model(eng)
    # h_g, h_s_g, c_s_g = decoder_model(ger_in, h_state, c_state)
    
    with tf.GradientTape() as tape: 
        h_s, c_s = encoder_model(eng)
        h_g, h_s, c_s = decoder_model(ger_in, h_s, c_s) # h_g: h_german
        
        loss_value = loss_func(ger_out, h_g) # ger_out: target, h_g: pred
        
    return loss_value, tape.gradient(loss_value, trainable_variables)

In [69]:
def train(num_epochs=10):
    train_loss = []
    val_loss = []
    
    for epoch in range(num_epochs):
        train_epoch_loss_avg = tf.keras.metrics.Mean()
        val_epoch_loss_avg = tf.keras.metrics.Mean()
        
#         test_ind = 0
        for eng, ger in train_dataset:
            ger_in, ger_out = german_in_out(ger)
            loss_value, grads= grad_fn(eng, ger_in, ger_out)
            
            optimizer.apply_gradients(zip(grads, trainable_variables))

            train_epoch_loss_avg(loss_value)
        
        for eng, ger in val_dataset:
            ger_in, ger_out = german_in_out(ger)
            loss_value, grads = grad_fn(eng, ger_in, ger_out)
            
            val_epoch_loss_avg(loss_value)
            
        train_loss.append(train_epoch_loss_avg.result())
        val_loss.append(val_epoch_loss_avg.result())
        
        print(" Epoch :{:03d} || Loss: {:.4f} || Val_Loss: {:.4f}".format(epoch, \
                                            train_epoch_loss_avg.result(), val_epoch_loss_avg.result()))
    return train_loss, val_loss

In [ ]:
train_loss, val_loss = train(num_epochs=10)

## 7. Use the model to translate
Now it's time to put your model into practice! You should run your translation for five randomly sampled English sentences from the dataset. For each sentence, the process is as follows:
* Preprocess and embed the English sentence according to the model requirements.
* Pass the embedded sentence through the encoder to get the encoder hidden and cell states.
* Starting with the special  `"<start>"` token, use this token and the final encoder hidden and cell states to get the one-step prediction from the decoder, as well as the decoder’s updated hidden and cell states.
* Create a loop to get the next step prediction and updated hidden and cell states from the decoder, using the most recent hidden and cell states. Terminate the loop when the `"<end>"` token is emitted, or when the sentence has reached a maximum length.
* Decode the output token sequence into German text and print the English text and the model's German translation.